In [1]:
import pickle
from scipy import sparse

In [2]:
with open('../data/idx_train.pckl', 'rb') as f:
    indices = pickle.load(f)

In [3]:
vectorizers = ['count', 'tfidf', 'hashing', 'binary', 'hashing_binary']

for vectorizer in vectorizers:
    for dataset in ['train', 'dev']:
        with open(f'../data/{dataset}_{vectorizer}_vectorized.pckl', 'rb') as f:
            vectorized_data = pickle.load(f)
        
        with open(f'../data/{dataset}_num_caps.pckl', 'rb') as f:
            caps = pickle.load(f)
        
        with open(f'../data/{dataset}_num_exclam.pckl', 'rb') as f:
            exclam = pickle.load(f)
            
        with open(f'../data/{dataset}_reviewsToDate.pckl', 'rb') as f:
            rev_counts = pickle.load(f)
            
        with open(f'../data/{dataset}_ratings.pckl', 'rb') as f:
            ratings = pickle.load(f)
        
        caps = caps.values.reshape(-1, 1)
        exclam = exclam.values.reshape(-1, 1)
        rev_counts = rev_counts.values.reshape(-1, 1)
        ratings = ratings.values.reshape(-1, 1)
        
        if dataset == 'train':
            # sample using the indices from `downsample.ipynb` / `upsample.ipynb`
            caps = caps[indices]
            exclam = exclam[indices]
            rev_counts = rev_counts[indices]
            ratings = ratings[indices]
        
        # more features
        
        full_data = sparse.hstack((vectorized_data,
                                   sparse.csr_matrix(caps),
                                   sparse.csr_matrix(exclam),
                                   sparse.csr_matrix(rev_counts),
                                   sparse.csr_matrix(ratings),
                                   # more features
                                   ))

        # for some reason, this isn't an actual CSR matrix...
        full_data = full_data.tocsr()

        with open(f'../data/{dataset}_{vectorizer}_subsampled_data.pckl', 'wb') as f:
            pickle.dump(full_data, f)